# Vorbereitung des Korpus

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
# vortrainierte Pipeline für das Italienische
nlp = spacy.load('it_core_news_lg')

In [3]:
lemmatizer = nlp.get_pipe('lemmatizer')
#stopwords = stopwords.words('italian')
#tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [4]:
df = pd.read_csv('Korpus/corpus_norm.csv', sep=',', encoding='utf-8')

In [5]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,filtered_text,weight
0,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv rime degli arcadi aglauro cidonia fau...,0.5
1,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv rime degli arcadi tirsi leucasio gio...,0.8
2,Poesia.IV.1.Testo.txt,MIDIA,Giuseppe Paolucci (Alessi Cillenio),Poesie,1700.0,1700-1750,poesia,IV. 1. Rime degli Arcadi: Alessi Cillenio (Giu...,10862.0,iv rime degli arcadi alessi cillenio giu...,0.8
3,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,iv lorenzo magalotti lettere odorose ...,0.8
4,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,iv ludovico antonio muratori antichità i...,0.5


In [6]:
df.shape

(155605, 11)

In [7]:
df.text = df.text.fillna('')

### Lemmatisierung

In [8]:
def lemmatize_text(raw_text):
    """
    lemmatizes the text
    """
    text = ''
    doc = nlp(raw_text)
    lemmas = ' '.join([x.lemma_ for x in doc])
    text = text + lemmas
    return text

In [9]:
nlp.max_length = 100000000000000            # Wert erhöhen, weil sehr großes Korpus...

In [10]:
%%time 
df['lemmatized_text'] = df['text'].apply(lemmatize_text)               

Wall time: 2h 15min 9s


In [11]:
# zwischenspeichern

df.to_csv('Korpus/corpus_lemmatized.csv', encoding='utf-8', index=False)

In [12]:
# neu laden 

df = pd.read_csv('Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [13]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,filtered_text,weight,lemmatized_text
0,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv rime degli arcadi aglauro cidonia fau...,0.5,iv . 4 . rima del arcadi : aglauro cidonia ( f...
1,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv rime degli arcadi tirsi leucasio gio...,0.8,iv . 11 . rima del arcade : tirsi leucasio ( g...
2,Poesia.IV.1.Testo.txt,MIDIA,Giuseppe Paolucci (Alessi Cillenio),Poesie,1700.0,1700-1750,poesia,IV. 1. Rime degli Arcadi: Alessi Cillenio (Giu...,10862.0,iv rime degli arcadi alessi cillenio giu...,0.8,iv . 1 . rima del arcadi : alessi cillenio ( G...
3,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,iv lorenzo magalotti lettere odorose ...,0.8,"iv . 5 . Lorenzo magalotti , lettere odoroso (..."
4,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,iv ludovico antonio muratori antichità i...,0.5,"﻿iv . 4 . Ludovico Antonio muratori , antichit..."


In [14]:
df['text'].iloc[60000]

"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Cesare Salvi , il membro della segreteria chiamato a presidiare Botteghe Oscure in questi giorni estivi ,  è  rimasto stupito dalla lettura dell' Unità di ieri .                                                                                                                                                                                                                                                                                                                                                      

In [15]:
df['lemmatized_text'].iloc[60000]

"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Cesare Salvi , il membro della segreteria chiamare a presidiare Botteghe oscuro in questo giorno estivo ,   essere   rimasto stupire dalla lettura dell ' unità di ieri .                                                                                                                                                                                                                                                                                                                                             

In [17]:
df['text'].iloc[2000]

'Simile armata di mare non avea mai fatta in addietro la potenza ottomana.  '

In [18]:
df['lemmatized_text'].iloc[2000]

'simile armata di mare non avea mai fare in addietro il potenza ottomano .  '

In [19]:
df['text'].iloc[2200]

' Ha principio un nuovo patriarcato, 1134.  '

In [20]:
df['lemmatized_text'].iloc[2200]

'  avere principio uno nuovo patriarcato , 1134 .  '